In [26]:
import datetime
import random
import warnings

import numpy as np
import pandas as pd
import plotapi
import plotly
import plotly.express as px
import plotly.graph_objects as go
import pyecharts.options as opts
from plotapi import SplitChord
from plotly.subplots import make_subplots
from pyecharts.charts import ThemeRiver
from tqdm import tqdm

plotly.io.templates.default = 'ggplot2'
plotapi.api_key('b0368160-7e02-4566-9dec-1829bd5b070b')

warnings.filterwarnings('ignore')

Your PlotAPI API key has been saved in your local environment. You will not need to set it again.


In [5]:
import os

# switch the working directory to project root
if os.getcwd().split('/')[-1] == 'code':
    os.chdir('..')
print(os.getcwd())


/Users/crinstaniev/Dev/ethereum/merge-data-challenge


## Theme River

In [6]:
river_df = pd.read_csv(
    'data/status.csv')[['eligible', 'activated', 'exited']].reset_index()
river_df['date'] = pd.date_range(
    start='2020-12-01 20:00:23', end='2022-10-31 00:00:00', periods=len(river_df)).floor('d')
river_df = river_df.groupby('date').max().astype(int).reset_index()

In [7]:
# construct dataset for plot
x_data = ['eligible', 'activated', 'exited']
y_data = []
item_last = None

for idx, item in tqdm(river_df.iterrows(), total=len(river_df)):

    y_data.append([str(item['date']).split(' ')[0],
                  item['eligible'], 'eligible'])
    y_data.append([str(item['date']).split(' ')[0],
                  item['activated'], 'activated'])
    y_data.append([str(item['date']).split(' ')[0], item['exited'], 'exited'])


100%|██████████| 700/700 [00:00<00:00, 20523.81it/s]


In [17]:
river = (
    ThemeRiver()
    .add(
        series_name=x_data,
        data=y_data,
        singleaxis_opts=opts.SingleAxisOpts(
            pos_top="50", pos_bottom="50", type_="time"
        ),
    )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="line")
    )
)

In [18]:
river.render("./figures/river.html")

'/Users/crinstaniev/Dev/ethereum/merge-data-challenge/figures/river.html'

In [19]:
river.render_notebook()

## Split Chord

In [20]:
block_df = pd.read_pickle('data/block_df.pkl')

In [21]:
block_mini_df = block_df.sort_values('block_slot')[:4]

com_df = pd.read_pickle('data/committees.pkl').reset_index()
com_df['slot'] = com_df['slot'].astype(int)
com_df['validators'] = com_df['validators'].apply(
    lambda x: random.sample(x, 32))
com_df = com_df.set_index('slot')
com_df = com_df.drop('index', axis=1)

In [22]:
# make nodes
slot_proposer_df = block_mini_df[['block_slot', 'proposer_index']]
slot_proposer_df.columns = ['slot', 'proposer_index']
slot_proposer_df['slot'] = slot_proposer_df['slot'].astype(int)
slot_proposer_df = slot_proposer_df.set_index('slot')

slot_info_df = slot_proposer_df.join(com_df, how='inner')

In [23]:
# construct links
links = []
for idx, item in tqdm(slot_info_df.iterrows(), total=len(slot_info_df)):
    proposer = item['proposer_index']
    for validator in item['validators']:
        links.append(dict(
            source=proposer,
            target=validator,
            value=1
        ))

nodes = []
rights = set()
lefts = set()
for item in links:
    lefts.add(item['source'])
    rights.add(item['target'])

for right in rights:
    nodes.append(dict(
        name=right,
        group='right'
    ))
for left in lefts:
    nodes.append(dict(
        name=left,
        group='left'
    ))

100%|██████████| 4/4 [00:00<00:00, 4175.51it/s]


In [33]:
fig = SplitChord(links, nodes)
fig.to_html('figures/chord.html')
fig.show()


## Slashing Status

In [34]:
df_blocktime = pd.read_csv('data/blocktime.csv')
df_status = pd.read_csv('data/status.csv')

df_status['date'] = pd.date_range(
    start='2020-12-01 20:00:23', end='2022-10-31 00:00:00', periods=len(df_status)).floor(freq='d')

df_status = df_status.groupby('date').max().reset_index()


In [35]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Line(
    x=df_status['date'],
    y=df_status['slashed'],
    name='slashed validators accumulated'
), secondary_y=False)

fig.add_vline(
    x=datetime.datetime.strptime("2022-09-15", "%Y-%m-%d").timestamp() * 1000,
    line_dash='dot',
    annotation_position="right",
    annotation_text="The Merge: Sep 15, 2022",
    line_color='black'
)

fig.add_trace(go.Line(
    x=df_status['date'],
    y=df_status['slashed'] - df_status['slashed'].shift(1),
    name='slashed validators'
), secondary_y=True)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label='1m', step='month', stepmode='backward'),
            dict(count=6, label='6m', step='month', stepmode='backward'),
            dict(count=1, label='YTD', step='year', stepmode='todate'),
            dict(count=1, label='1y', step='year', stepmode='backward'),
            dict(step='all')
        ])
    )
)

fig.show()
fig.write_html('figures/slashing_stats.html')

/Users/crinstaniev/.local/share/virtualenvs/merge-data-challenge-ju38e7AG/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




## Blocktime Data

In [36]:
df_blocktime = pd.read_csv('data/blocktime.csv', parse_dates=['Date(UTC)'])
df_blocktime = df_blocktime[df_blocktime['Date(UTC)'] > '2020-12-01']

In [37]:
fig = make_subplots()

fig.add_trace(go.Scatter(
    x=df_blocktime['Date(UTC)'],
    y=df_blocktime['Value']
))

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label='1m', step='month', stepmode='backward'),
            dict(count=6, label='6m', step='month', stepmode='backward'),
            dict(count=1, label='YTD', step='year', stepmode='todate'),
            dict(count=1, label='1y', step='year', stepmode='backward'),
            dict(step='all')
        ])
    )
)

fig.show()

fig.write_html('figures/blocktime.html')